In [1]:
import pandas as pd
from eval import my_grid_search_cv
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

# from tqdm import tqdm
# tqdm.pandas()

In [2]:
import os

if os.getcwd() != "/root/restaurants":
    os.chdir("..")
os.getcwd()

'/root/restaurants'

In [3]:
data = pd.read_csv("data/processed/data.csv")
data.head()

data["date"] = pd.to_datetime(data["date"])
data = data.rename(columns={"date": "visit_date"})
data.head()


,store_id,visit_date,day_of_week,holiday_flg,genre_name,area_name,latitude,longitude,year,month,...,is_nan_lag_holiday_flg_273,lag_area_genre_day_of_week_365,is_nan_lag_area_genre_day_of_week_365,visitors,lag_area_genre_holiday_flg_365,is_nan_lag_area_genre_holiday_flg_365,lag_day_of_week_365,is_nan_lag_day_of_week_365,lag_holiday_flg_365,is_nan_lag_holiday_flg_365
0,air_00a91d42b08b08d9,2016-07-01,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,...,1,0.0,1,24,0.0,1,0.0,1,0.0,1
1,air_0241aa3964b7f861,2016-01-03,Sunday,1,Izakaya,Tōkyō-to Taitō-ku Higashiueno,35.712607,139.779996,2016,1,...,1,0.0,1,28,0.0,1,0.0,1,0.0,1
2,air_034a3d5b40d5b1b1,2016-07-01,Friday,0,Cafe/Sweets,Ōsaka-fu Ōsaka-shi Ōhiraki,34.692337,135.472229,2016,7,...,1,0.0,1,9,0.0,1,0.0,1,0.0,1
3,air_036d4f1ee7285390,2016-07-01,Friday,0,Cafe/Sweets,Hyōgo-ken Takarazuka-shi Tōyōchō,34.799767,135.360073,2016,7,...,1,0.0,1,10,0.0,1,0.0,1,0.0,1
4,air_04341b588bde96cd,2016-01-01,Friday,1,Izakaya,Tōkyō-to Nerima-ku Toyotamakita,35.735623,139.651658,2016,1,...,1,0.0,1,12,0.0,1,0.0,1,0.0,1


In [7]:
import numpy as np
num_cols = np.array(X.select_dtypes(include=['number']).columns)

In [7]:
param_grid = {        
    "n_estimators": [20, 30, 40, 50, 70, 100, 200, 300],
    "learning_rate": [0.01, 0.05, 0.1, 0.5],
    "max_depth": [3, 4, 5],

    "eval_metric": ["rmsle"],
    "enable_categorical": ["True"],
}

params = my_grid_search_cv(XGBRegressor, X, y, param_grid, n_jobs=1)

model = XGBRegressor(**params)

X = X.drop(columns=["visit_date"]).reset_index(drop=True)

scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

model.fit(X, y)

  0%|          | 0/84 [00:00<?, ?it/s]

 17%|█▋        | 14/84 [01:16<06:46,  5.81s/it]

: 

In [7]:
directory = "models/xgboost"
os.makedirs(directory, exist_ok=True)
model.save_model(directory + '/xgboost_.json')

NameError: name 'model' is not defined

In [8]:
THREE_WEEKS = 21
FIVE_WEEKS = 35
TWO_MONTH = 61
ONE_QUARTER = 365 // 4
HALF_YEAR = 365 // 2
THREE_QUARTERS = 365 * 3 // 4
YEAR = 365

LAGS = [
    THREE_WEEKS,
    FIVE_WEEKS,
    TWO_MONTH,
    ONE_QUARTER,
    HALF_YEAR,
    THREE_QUARTERS,
    YEAR,
]


In [7]:
model = XGBRegressor()
model.load_model("models/xgboost/xgboost_more_estimators.json")
# import joblib
# model = joblib.load('models/xgboost/xgboost_more_estimators.pkl')


In [9]:
from tqdm.auto import tqdm

In [10]:
ID_SIZE = 20

store_info = pd.read_csv("data/raw/air_store_info.csv")
store_info = store_info.rename(
    columns={
        "air_store_id": "store_id",
        "air_genre_name": "genre_name",
        "air_area_name": "area_name",
    }   
)

date_info = pd.read_csv("data/raw/date_info.csv")
date_info = date_info.rename(columns={"calendar_date": "date"})
date_info["date"] = date_info["date"].astype("string")

X = data.drop(columns=["visitors"]).reset_index(drop=True)
num_cols = np.array(X.select_dtypes(include=['number']).columns)

scaler = StandardScaler()
scaler.fit(X[num_cols])


StandardScaler()

In [11]:
from copy import deepcopy

submission = pd.read_csv("data/raw/sample_submission.csv")
data = pd.read_csv("data/processed/data.csv")
data["date"] = pd.to_datetime(data["date"])
data.head()


new_submission = deepcopy(submission)
new_submission["store_id"] = new_submission["id"].str[:ID_SIZE]
new_submission["date"] = new_submission["id"].str[ID_SIZE + 1:]

new_submission = pd.merge(new_submission, date_info, on="date")
new_submission = pd.merge(new_submission, store_info, on="store_id")

new_submission["date"] = pd.to_datetime(new_submission["date"])
new_submission["year"] = new_submission["date"].dt.year
new_submission["month"] = new_submission["date"].dt.month
new_submission["day"] = new_submission["date"].dt.day
new_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32019 entries, 0 to 32018
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           32019 non-null  object        
 1   visitors     32019 non-null  int64         
 2   store_id     32019 non-null  object        
 3   date         32019 non-null  datetime64[ns]
 4   day_of_week  32019 non-null  object        
 5   holiday_flg  32019 non-null  int64         
 6   genre_name   32019 non-null  object        
 7   area_name    32019 non-null  object        
 8   latitude     32019 non-null  float64       
 9   longitude    32019 non-null  float64       
 10  year         32019 non-null  int32         
 11  month        32019 non-null  int32         
 12  day          32019 non-null  int32         
dtypes: datetime64[ns](1), float64(2), int32(3), int64(2), object(5)
memory usage: 2.8+ MB


In [12]:
import math

def add_lag_last(df, lag, column):
    column_name = f"lag_{column}_{lag}"
    is_nan_column_name = f"is_nan_{column_name}"

    last_row = df.iloc[-1]
    store_id = last_row["store_id"]
    column_value = last_row[column]
    start_date = last_row["date"]
    end_date = start_date - pd.Timedelta(days=lag)

    temp = df.loc[
        (df["store_id"] == store_id) &
        (df[column] == column_value) &
        (df['date'] < start_date) &
        (df['date'] >= end_date),
        "visitors"
    ]

    value = temp.mean()

    if pd.isna(value):
        value = 0
        is_nan = 1
    else:
        is_nan = 0
    
    df.at[df.index[-1], column_name] = value
    df.at[df.index[-1], is_nan_column_name] = is_nan

    return df

def get_area_genre_lag_last(df, lag, column):
    column_name = f"lag_{column}_{lag}"
    is_nan_column_name = f"is_nan_{column_name}"
    
    last_row = df.iloc[-1]
    area_name = last_row["area_name"]
    genre_name = last_row["genre_name"]
    column_value = last_row[column]
    start_date = last_row["date"]
    end_date = start_date - pd.Timedelta(days=lag)


    area_genre_data = df.loc[
        (df["area_name"] == area_name) &
        (df["genre_name"] == genre_name) &
        (df['date'] < start_date) & 
        (df['date'] >= end_date) & 
        (df[column] == column_value),
        ["date", "visitors", column]
    ]


    area_genre_data_mean = area_genre_data.groupby(by=["date"], as_index=False).visitors.mean()

    value = area_genre_data_mean.mean()["visitors"]

    if pd.isna(value):
        value = 0
        is_nan = 1
    else:
        is_nan = 0
    
    df.at[df.index[-1], column_name] = value
    df.at[df.index[-1], is_nan_column_name] = is_nan

    return df


In [62]:
def make_prediction(row):
    global data
    new_row = pd.DataFrame(row).T
    new_row = new_row.drop(columns="id")
    new_row["date"] = pd.to_datetime(new_row["date"])

    data = pd.concat([data, new_row]).reset_index(drop=True)

    for lag in LAGS:
        data = get_area_genre_lag_last(data, lag, "day_of_week")
        data = get_area_genre_lag_last(data, lag, "holiday_flg")
        data = add_lag_last(data, lag, "day_of_week")
        data = add_lag_last(data, lag, "holiday_flg")

    new_row = data.tail(1)

    new_row = new_row.drop(columns=["date", "visitors"])

    for c in new_row.columns:
        col_type = new_row[c].dtype
        if col_type == 'object' or col_type.name == 'category':
            new_row[c] = new_row[c].astype('category')

    new_row[num_cols] = scaler.transform(new_row[num_cols])

    pred = model.predict(new_row)
    pred = max(0, pred)

    row["visitors"] = pred
    data.at[data.index[-1], "visitors"] = pred
   
    return row

In [ ]:
%%prun
new_submission.head(6).progress_apply(lambda row: make_prediction(row), axis=1)

In [ ]:
import math
from pathlib import Path

directory = "data/submissions"
os.makedirs(directory, exist_ok=True)

num_parts = math.ceil(len(new_submission) / 1000)

for i in tqdm(range(num_parts), desc="Processing batches"):
    batch = new_submission.iloc[i*1000:(i+1)*1000]
    batch = batch.progress_apply(lambda row: make_prediction(row), axis=1)

    if i == 0:
        new_submission_processed = batch
    else:
        new_submission_processed = pd.concat([new_submission_processed, batch])
    
    output_path = Path(directory).joinpath(f"/new_submission_processed_{i}.csv")
    new_submission_processed.to_csv(output_path, index=False)

output_path = Path(directory).joinpath("xgboost_without_leakage.csv")
new_submission_processed.to_csv(output_path, index=False)

In [13]:
new_submission

,id,visitors,store_id,date,day_of_week,holiday_flg,genre_name,area_name,latitude,longitude,year,month,day
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,Sunday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2017,4,23
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,Monday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2017,4,24
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,Tuesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2017,4,25
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,Wednesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2017,4,26
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,Thursday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2017,4,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32014,air_fff68b929994bfbd_2017-05-27,0,air_fff68b929994bfbd,2017-05-27,Saturday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2017,5,27
32015,air_fff68b929994bfbd_2017-05-28,0,air_fff68b929994bfbd,2017-05-28,Sunday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2017,5,28
32016,air_fff68b929994bfbd_2017-05-29,0,air_fff68b929994bfbd,2017-05-29,Monday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2017,5,29
32017,air_fff68b929994bfbd_2017-05-30,0,air_fff68b929994bfbd,2017-05-30,Tuesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2017,5,30


In [ ]:
new_submission.groupby(by=["date"]).transform()

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()
new_submission = new_submission.progress_apply(lambda row: make_prediction(row), axis=1)

In [90]:
new_submission_processed["visitors"] = new_submission_processed.visitors.astype(float)
new_submission_processed["visitors"].unique()

array([21.16773605])

In [ ]:
new_submission_processed.to_csv("xgb_new_submission_processed.csv", index=False)

In [15]:
directory = "data/submissions"
os.makedirs(directory, exist_ok=True)

new_submission = new_submission[["id", "visitors"]]
new_submission.to_csv(directory + "/xgboost_more_estimators.csv", index=False)